<a href="https://colab.research.google.com/github/BlackCurrantDS/DBSE_Project/blob/main/compute_confidence_rule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
minconf = 0.8

In [191]:
#reading original data
df = pd.read_csv("/content/drive/MyDrive/DataSets_for_Paper/bcancer_data_original.csv")

In [192]:
df

,Unnamed: 0,class,AGE,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no
...,...,...,...,...,...,...,...,...,...,...,...
281,281,recurrence-events,30-39,premeno,30-34,0-2,no,2,left,left_up,no
282,282,recurrence-events,30-39,premeno,20-24,0-2,no,3,left,left_up,yes
283,283,recurrence-events,60-69,ge40,20-24,0-2,no,1,right,left_up,no
284,284,recurrence-events,40-49,ge40,30-34,3-5,no,3,left,left_low,no


In [265]:
#to pass into query
df.rename(columns={'breast-quad':'quad'}, inplace=True)
df.rename(columns={'class':'target'}, inplace=True)
df.rename(columns={'tumor-size':'tumor'}, inplace=True)
df.rename(columns={'inv-nodes':'inv'}, inplace=True)
df.rename(columns={'node-caps':'node'}, inplace=True)
df.rename(columns={'deg-malig':'malig'}, inplace=True)


In [193]:
total_transactions = len(df)

286

In [ ]:
#reading rule file

In [354]:
import fileinput

text = "/content/output_rules_final_01.txt"
fields = {">": ",", 'breast-quad':'quad', 'class':'target','tumor-size':'tumor',
          'inv-nodes':'inv','node-caps':'node','deg-malig':'malig', '"': ''}


for line in fileinput.input(text, inplace=True):
    line = line.rstrip()
    if not line:
        continue
    for f_key, f_value in fields.items():
        if f_key in line:
            line = line.replace(f_key, f_value)
    print(line)

In [355]:
crimefile = open(text, 'r')
yourResult = [line.split(',') for line in crimefile.readlines()]

In [356]:
"""rules = [
         
     ['malig@3','target@recurrence-events','irradiat@no','menopause@ge40','breast@left','quad@left_low']    ,
     ['tumor@50-54','breast@right','target@recurrence-events']
]"""

"rules = [\n         \n     ['malig@3','target@recurrence-events','irradiat@no','menopause@ge40','breast@left','quad@left_low']    ,\n     ['tumor@50-54','breast@right','target@recurrence-events']\n]"

In [358]:
myfile = open('rpp_longest_rules_class_yes_confidence.txt', 'w')
for rule in yourResult:
    #print(rule)
    list_len = len(rule) #end of the list
    q=[]
    for _, i in enumerate(rule):
      #print(_)
      i = i. rstrip("\n")
      s=i.split("@")
      att = s[0]
      val = s[1]
      
      if _ == list_len-1:
              if type(val) == int:
                #v.append("(df"+"['"+att+"']=="+val+")")
                q.append(att+"=="+val+"")
              else:
                #v.append("(df"+"['"+att+"']=='"+val+"')")
                q.append(att+"=='"+val+"'")
      elif type(val) == int:
        #print(type(val))
        #v.append("(df"+"['"+att+"']=="+val+") &")
        q.append(att+"=="+val+" & ")
      else:
        #v.append("(df"+"['"+att+"']=='"+val+"') & ")
        q.append(att+"=='"+val+"' & ")
    
    #print(q)
    
    
    s = ''.join(q)

    #print("s without removing class", s)
    #string without class
    without = s.replace("& target=='recurrence-events'", "")
    #print("wihtout",without)

    s = s.replace("'", '"')
    without = without.replace("'", '"')
    #print("for rule {}, confidence is {}, string is {}".format(rule,len(df.query(s)), s))
    #print("for wihtout {}, confidence is {}".format(without,len(df.query(without))))
    total = len(df.query(s))
    left = len(df.query(without))
    if total == 0 or left ==0:
      confidence = 0
    else:
      confidence = round(total/left,3)
    #print("for rule {} confidence is {}".format(rule, confidence))
    
    if confidence >= .80:
      myfile.writelines("Rule = {} confidence ={}".format(rule, confidence)+'\n')
myfile.close()
